# High Speed E-Stop CCW/Rotator Integration Test

This notebook performs a high speed E-Stop interlock scenario integration test between the Camera Cable Wrap (CCW) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [1]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import MTPtg

from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime

In [2]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

In [3]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [4]:
test_message = "E-Stop MTPtg_Rotator_CCW Integration Test"

In [5]:
d = salobj.Domain()

In [6]:
script = salobj.Controller("Script", index=1)

In [7]:
rot = salobj.Remote(d, "MTRotator")

In [8]:
mtptg = salobj.Remote(d, "MTPtg")

In [9]:
mtm = salobj.Remote(d, "MTMount")

In [10]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

rotation DDS read queue is filling: 99 of 100 elements
motors DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is full (100 elements); data may be lost
ccwFollowingError DDS read queue is full (100 elements); data may be lost
timeAndDate DDS read queue is filling: 12 of 100 elements
mountStatus DDS read queue is filling: 12 of 100 elements
mountPosition DDS read queue is filling: 13 of 100 elements
currentTargetStatus DDS read queue is filling: 13 of 100 elements


[None, None, None, None]

In [32]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [35]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [13]:
await salobj.set_summary_state(mtm, salobj.State.ENABLED)

[<State.ENABLED: 2>]

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.OFFLINE)

# Move to 0 deg Starting Position

In [17]:
print("Move to 0.0 deg starting position")

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
print ("curr_time_mtptg", curr_time_mtptg)
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = rot_tel
print("rot_pa", rot_pa)

#await rot.cmd_stop.start(timeout=10)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

Move to 0.0 deg starting position
curr_time_mtptg 59382.957857431924
Time error=-0.00 sec
Alt: 45.0 deg
Az: 0.0 deg
RotTel: 0.0 deg
ParaAng: 179.67215364333862 deg
rot_pa 0d00m00s


In [22]:
#await asyncio.sleep(30.)
await mtptg.cmd_stopTracking.start(timeout=10)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2059903306, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [31]:
await rot.cmd_stop.start(timeout=10)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1893552169, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Not enabled')

In [28]:
await mtm.cmd_stop.start(timeout=10)

# Start Test

In [ ]:
now = datetime.now()
print(now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message}")

# Send Track Command to +60 deg

In [25]:
print("Azimuth = 0 and rot_tel= 60 deg")

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(60, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")

# Calculating the other parameters     
#rot_pa = para_ang-180*u.deg+rot_tel
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

Azimuth = 0 and rot_tel= 60 deg
Time error=-0.00 sec
Alt: 45.0 deg
Az: 0.0 deg
RotTel: 60.0 deg
ParaAng: 179.67240154475238 deg
60d00m00s


# For Testing - This is pushing E-Stop

In [ ]:
await asyncio.sleep(10.)
await mtptg.cmd_stopTracking.start(timeout=30)
await rot.cmd_stop.start(timeout=30)
await mtm.cmd_stop.start(timeout=30)

# Test Complete

In [ ]:
print("Test Complete")
now = datetime.now()
print(now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                          message=f"END - {test_message}")

# Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

Wait for override off

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [27]:
await mtm.cmd_enterControl.start(timeout=10.)
await mtm.cmd_start.start(timeout=10.)
await mtm.cmd_enable.start(timeout=10.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=842477643, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Not supported by this CSC')

# Move to sync position with CCW

In [36]:
print("Move to X.X deg starting position")

await rot.cmd_move.set_start(position=-10.02,timeout=30.)

Move to X.X deg starting position


# Additional Error Handling Commands

In [34]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

In [ ]:
await mtptg.cmd_clearError.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)

In [ ]:
await rot.cmd_start.start(timeout=10.)

In [ ]:
await rot.cmd_enable.start(timeout=10.)